### **Scraping del sitio web con scrolling infinito**

Por lo visto Twitter sólo carga los tweets de la página en la que estamos en ese momento. Así que esto significa que sólo obtendremos los tweets de la página en la que estemos en el momento en que hagamos el scraping. Es decir, tendremos que scrapear cada página a medida que scrolleamos hacia abajo y no esperar hasta el final para scrapearlo todo. Desafortunadamente, una de las desventajas de este enfoque es que podríamos obtener tweets duplicados porque Selenium podría capturar elementos que ya ha localizado de las páginas anteriores. Vamos a manejar estos tweets duplicados con sets (conjuntos).

Es decir, si buscamos un usuario en la pagina del tweet que estamos trabajando, al principio si lo encuentra. Pero, a medida que scrolleamos hacia abajo (y se va cargando la página) ya no encuentra a ese usuario.

<center><img src="https://i.postimg.cc/3JxbdYsg/ws-33.png"></center>
<center><img src="https://i.postimg.cc/ncSSmNxN/ws-34.png"></center>

Diseño de código:

In [ ]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

website = 'https://twitter.com/Support/status/1600561455488970752'
driver = webdriver.Chrome(executable_path = r'C:\Users\Alfonso\Downloads\Programas\chromedriver\chromedriver.exe')
driver.get(website)
driver.maximize_window()

def get_tweet(element):
    """Esta función scrapea datos de tweets. Devuelve una lista con 2 elementos; nombre de usuario y texto"""
    try:
        user = element.find_element_by_xpath(".//span[contains(text(), '@')]").text  # Obtener el nombre de usuario
        text = element.find_element_by_xpath(".//div[@lang]").text  # Obtener el texto de la tarjeta
        tweets_data = [user, text]
    except:
        tweets_data = ['user', 'text']
    return tweets_data


user_data = []
text_data = []
tweet_ids = set()
scrolling = True
while scrolling:
    tweets = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, "//article[@role='article']")))
    print(len(tweets))
    for tweet in tweets[-15:]:  # puedes cambiar este número con el número de tweets en un sitio web || NOTA: SÓLO SE CONSIDERARÁN 
                                # AQUELLOS CARGADOS EN LA ÚLTIMA PÁGINA mientras que se olvidarán los de la página anterior (ejemplo: 
                                # desplácese hasta abajo y luego intente encontrar un @nombredeusuario que esté arriba --> no lo encontrará)
        tweet_list = get_tweet(tweet)
        tweet_id = ''.join(tweet_list)
        if tweet_id not in tweet_ids:
            tweet_ids.add(tweet_id)
            user_data.append(tweet_list[0])
            text_data.append(" ".join(tweet_list[1].split()))

    # Obtener la altura inicial del scroll
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:  # Siempre se ejecutará este loop hasta al momento que sea "False" o se ejecute el "break"
        # Scroll hacia abajo hasta la parte inferior
        # Este argumento "document.body.scrollHeight" nos indica la tamaño(size)/altura(height) de la página. Si escribimos este comando
        # nos servirá para cualquier pantalla (1080, etc.)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Esperar para cargar la página
        time.sleep(5)
        # Calcular la nueva altura del scroll y compararla con la última altura del scroll
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:  # si la altura nueva y la última son iguales, significa que no hay ninguna página nueva que cargar, así que dejamos de hacer scroll
            scrolling = False
            break
        # condition 2
        # if len(data) > 60:
        #     scrolling = False
        #     break
        else:
            last_height = new_height
            break

driver.quit()

df_tweets = pd.DataFrame({'user': user_data, 'text': text_data})
df_tweets.to_csv('tweets_pagination.csv', index=False)
print(df_tweets)

In [4]:
abecedario = 'abcdefghijklmnopqrstuvwxyz'

for letra in abecedario[-5:]:
    print(letra)

v
w
x
y
z


En resumen, esto es lo que hace el código:

1. Ingresa al sitio web especifico de un tweet
2. Va a tomar los últimos 15 comentarios (podemos modificar este número a gusto)
3. Luego para cada comentario se crea un ID, uniendo todo el usuario y texto
4. Luego, para evitar la duplicacion de un comentario, se utiliza un IF para ir comparando los ID
5. Si no, existe se agrega a un conjunto (no permite duplicion de un elemento)
6. Luego, se agrega la data en user_data y text_data

<center><img src="https://i.postimg.cc/6Q3BDvj2/ws-35.png"></center>

7. Y luego scrolea hacia al final de la página y se cargan nuevos comentarios
8. Si mantiene el mismo tamaño/altura la página se rompen el primer y segundo while y genera el dataframe y el archivo CSV
9. Si no mantiene el mismo tamaño/altura la página (se cargan nuevos comentarios) nos saca del segundo while 
10. Se repiten los pasos del 2 al 6

<center><img src="https://i.postimg.cc/HnGHcPpX/ws-36.png"></center>